# Agentic RAG Using CrewAI & LangChain

We are going to see how agents can be involved in the RAG system to rtrieve the most relevant information.

In [32]:
from langchain_openai import ChatOpenAI
import os
from crewai_tools import PDFSearchTool
from langchain_community.tools.tavily_search import TavilySearchResults
from crewai_tools  import tool
from crewai import Crew,LLM, Process
from crewai import Task
from crewai import Agent
from dotenv import load_dotenv

load_dotenv()


True

## Mention the LLM being used

In [49]:
llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.7,
)

# llm = LLM(
#     model="ollama/llama3.2",
#     base_url="http://localhost:11434"
# )

## Create a RAG tool variable to pass our PDF

In [50]:
rag_tool = PDFSearchTool(pdf='ML.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        # llm=dict(
        #     provider="ollama", # or google, openai, anthropic, llama2, ...
        #     config=dict(
        #         model="llama3.2",
        #         # temperature=0.5,
        #         # top_p=1,
        #         # stream=true,
        #     ),
        # ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

In [51]:
# res = rag_tool.run("what is transformer?")
# print(res)

In [53]:
from crewai_tools import SerperDevTool

# Initialize the tool for internet searching capabilities
web_search_tool = SerperDevTool()

## Create Agents to work with

In [56]:
retriever_agent = Agent(
    role="""Retrieve relevant information to answer the user query: {query}""",
    goal="""Retrieve the most relevant information from the available sources 
            for the user query: {query}, always try to use the rag tool first. 
            If you are not able to retrieve the information from the pdf search tool 
            then try to use the web search tool.""",
    backstory="""You're a meticulous analyst with a keen eye for detail. 
                You're known for your ability understand the user query: {query} 
                and retrieve knowlege from the most suitable knowledge base.""",
    verbose=True,
    tools=[
        rag_tool,
        web_search_tool
    ],
    llm=llm
)

response_synthesizer_agent = Agent(
    role="""Response synthesizer agent for the user query: {query}""",
    goal="""Synthesize the retrieved information into a concise and coherent response 
            based on the user query: {query}. If you are not able to retrieve the 
            information then respond with "I'm sorry, I couldn't find the information 
            you're looking for.""",
    backstory="""You're a skilled communicator with a knack for turning complex 
                information into clear and concise responses.""",
    verbose=True,
    llm=llm
)


In [57]:
retrieval_task = Task(
    description="""Retrieve the most relevant information from the available 
                   sources for the user query: {query}""",
    expected_output="""The most relevant information in form of text as retrieved
                       from the sources.""",
    agent=retriever_agent
)

response_task = Task(
    description="""Synthesize the final response for the user query: {query}""",
    expected_output="""A concise and coherent response based on the retrieved infromation
                       from the right source for the user query: {query}. If you are not 
                       able to retrieve the information then respond with 
                       I'm sorry, I couldn't find the information you're looking for.""",
    agent=response_synthesizer_agent
)

In [58]:
crew = Crew(
			agents=[retriever_agent, response_synthesizer_agent], 
			tasks=[retrieval_task, response_task],
			process=Process.sequential,
			# verbose=True,
			# process=Process.hierarchical, # In case you wanna use that instead https://docs.crewai.com/how-to/Hierarchical/
		)

2025-01-18 13:57:33,327 - 20468 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [ ]:
result = crew.kickoff(inputs={"query": "what is machine learning"})

# Agent: Retrieve relevant information to answer the user query: what is machine learning
## Task: Retrieve the most relevant information from the available 
                   sources for the user query: what is machine learning


# Agent: Retrieve relevant information to answer the user query: what is machine learning
## Using tool: Search a PDF's content
## Tool Input: 
"{\"query\": \"what is machine learning\"}"
## Tool Output: 
Relevant Content:
1. Introduction to Machine Learning 1.1 Definition and importance of ML : Machine learning is a branch of artificial intelligence (AI) and computer science which focuses on the use of data and algorithms to imitate the way that humans learn, gradually improving its accuracy. 1.2 Types of machine learning: a) Supervised Learning Supervised learning is a type of machine learning where the algorithm is trained on labeled data. This means that each training example is paired with an output label. The goal is for the model to learn to map input

In [46]:
print(result)

The current Prime Minister of India is Narendra Modi. He was sworn-in as India's Prime Minister for the third time on 9th June 2024, following another decisive victory in the 2024 Parliamentary elections. For more information, you can visit the official website of the Prime Minister of India at http://www.pmindia.gov.in/en/ or the Wikipedia page of the Prime Minister of India at https://en.wikipedia.org/wiki/Prime_Minister_of_India.
